# Recommendation System

In [ ]:
pip uninstall numpy

In [19]:
import pandas as pd
import numpy as np

## Item based collaborative system

In [3]:
data = pd.read_json('renttherunway_final_data.json', lines=True, nrows=10000)

In [4]:
data.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,"5' 4""",4,116.0,"December 14, 2015"
3,fit,909926,34c,126335,135lbs,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34.0,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"


In [5]:
filtered_data = data[(data['age'] >= 18) & (data['age'] <= 35)]


In [6]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6918 entries, 0 to 9997
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fit             6918 non-null   object 
 1   user_id         6918 non-null   int64  
 2   bust size       6261 non-null   object 
 3   item_id         6918 non-null   int64  
 4   weight          5783 non-null   object 
 5   rating          6918 non-null   int64  
 6   rented for      6917 non-null   object 
 7   review_text     6918 non-null   object 
 8   body type       6411 non-null   object 
 9   review_summary  6918 non-null   object 
 10  category        6918 non-null   object 
 11  height          6890 non-null   object 
 12  size            6918 non-null   int64  
 13  age             6918 non-null   float64
 14  review_date     6918 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 864.8+ KB


In [7]:
nan_count_per_column = filtered_data.isna().sum()
print("\nNumber of NaN values in each column:")
print(nan_count_per_column)


Number of NaN values in each column:
fit                  0
user_id              0
bust size          657
item_id              0
weight            1135
rating               0
rented for           1
review_text          0
body type          507
review_summary       0
category             0
height              28
size                 0
age                  0
review_date          0
dtype: int64


In [8]:
filtered_data = filtered_data.dropna()


In [9]:
filtered_data.shape

(5215, 15)

In [10]:
filtered_data.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016"
3,fit,909926,34c,126335,135lbs,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34.0,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"
6,fit,336066,34c,568429,112lbs,10,everyday,This dress is so sweet. I loved the print. The...,hourglass,LITERALLY THE CUTEST DRESS EVER,dress,"5' 3""",4,27.0,"December 7, 2017"
9,fit,154309,32b,1729232,114lbs,10,formal affair,The dress was very flattering and fit perfectl...,petite,This dress was everything! It was perfect for ...,gown,"5' 3""",1,33.0,"October 17, 2016"


user_id  item_id  rating

In [11]:
filtered_data = filtered_data[['user_id', 'item_id', 'rating']]

In [12]:
from scipy.sparse import csr_matrix

n_users =  filtered_data["user_id"].nunique()
n_items =  filtered_data["item_id"].nunique()
n_items = filtered_data['item_id'].max()


In [13]:
# Clean up unnecessary variables to free memory and improve performance
keep = ['filtered_data', 'data', 'data_cleaned','n_users','n_items',]
for var in list(globals()):
    if var not in keep and not var.startswith("__"):
        del globals()[var]

In [14]:
print(n_users)
print(n_items)

4746
2963850


In [15]:
print(filtered_data.head())
print(filtered_data.describe())


   user_id  item_id  rating
0   420272  2260466      10
3   909926   126335       8
4   151944   616682      10
6   336066   568429      10
9   154309  1729232      10
             user_id       item_id       rating
count    5215.000000  5.215000e+03  5215.000000
mean   503950.886481  1.029934e+06     9.116395
std    292353.761647  7.945314e+05     1.395947
min       321.000000  1.233730e+05     2.000000
25%    251276.000000  1.880520e+05     8.000000
50%    508011.000000  9.388820e+05    10.000000
75%    754087.000000  1.662059e+06    10.000000
max    999910.000000  2.963850e+06    10.000000


In [16]:
A = np.zeros((n_users,n_items))
for line in filtered_data.itertuples():
    A[line[1]-1,line[2]-1] = line[3]
print("Original rating matrix : ",A)

NameError: name 'np' is not defined

In [ ]:


#sparse_matrix list
rows = filtered_data["user_id"].values
cols = filtered_data["item_id"].values
data = np.where(filtered_data["rating"] >= 3, 1, 0) 

sparse_matrix = csr_matrix((data, (rows, cols)), shape=(n_users,n_items))

In [ ]:
n_users = filtered_data.user_id.unique().shape[0]
n_items = filtered_data.item_id.unique().shape[0]
n_items = filtered_data['item_id'].max()
A = np.zeros((n_users,n_items))
for line in dataset.itertuples():
    A[line[1]-1,line[2]-1] = line[3]
print("Original rating matrix : ",A)

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


data = data.dropna(subset=['rating', 'user_id', 'item_id'])

# Crear matriz de utilidad (usuario-ítem)
utility_matrix = data.pivot_table(index='item_id', columns='user_id', values='rating', fill_value=0)

# Calcular la similitud entre ítems
item_similarity = cosine_similarity(utility_matrix)
item_similarity_df = pd.DataFrame(item_similarity, index=utility_matrix.index, columns=utility_matrix.index)

# Ejemplo de ítems similares
item_id = 123  # Cambiar por un ID válido
similar_items = item_similarity_df[item_id].sort_values(ascending=False).head(5)
print(similar_items)


In [ ]:
n_items

In [ ]:
data.describe()

In [ ]:
counts = data['item_id'].value_counts()
counts

In [ ]:
para

In [ ]:
data.head()

# Market basket analysis

In [ ]:
data = pd.read_csv('bread basket.csv')

In [ ]:
data.head()

In [ ]:
data = data.drop(['date_time','period_day','weekday_weekend'], axis=1)

In [ ]:
data

In [ ]:
data = data.groupby('Transaction')['Item'].apply(list)
data

In [ ]:
data.info()

In [ ]:
transactions = data.tolist()
transactions

In [ ]:
items = list(set(item for transaction in transactions for item in transaction))

items

In [ ]:
encoded_data = pd.DataFrame(
    [[item in transaction for item in items] for transaction in transactions],
    columns=items
)


encoded_data.head()

# Apriori Algorithm

In [ ]:
import time

start_time = time.time()

frequent_itemsets = apriori(encoded_data, min_support=0.01, use_colnames=True)
frequent_itemsets

# End timing
end_time = time.time()

# Calculate the time taken
calculation_time = end_time - start_time

print("Frequent Itemsets calculated in {:.2f} seconds.".format(calculation_time))

In [ ]:
rules = association_rules(frequent_itemsets, metric = "antecedent support", min_threshold = 0.2)
rules.info()

In [ ]:
# Display the results
from IPython.display import display

print("Frequent Itemsets:")
display(frequent_itemsets.head())

In [ ]:
print("\nAssociation Rules:")
rules

In [ ]:
print(encoded_data['Bread'].sum())
print(encoded_data['Coffee'].sum())

# FP Growth Algorithm

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
start_time = time.time()


# Evaluate the frequent item rules for fpgrowth function
frequent_itemsets_fp = fpgrowth(encoded_data, min_support = 0.01, use_colnames = True)

print(frequent_itemsets_fp)

# End timing
end_time = time.time()

# Calculate the time taken
calculation_time = end_time - start_time

print("Frequent Itemsets calculated in {:.2f} seconds.".format(calculation_time))

In [ ]:
from mlxtend.frequent_patterns import association_rules

# Display the reles due to fp-growth algorithm
rules_fp = association_rules(frequent_itemsets_fp, metric = "confidence", min_threshold = 0.3)

rules_fp